In [ ]:
#이미지가 두 개이상인 경우도 맨 처음 이미지만 수집됩니다.
#해당 게시물이 영상이어도, 페이지 내에 뜨는 다른 게시물의 첫 번째 이미지가 수집됩니다...
#댓글에 있는 해시태그도 다 수집되는 것 같아요.

In [1]:
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [11]:
path = 'C:/Users/ASUX/Desktop/chromedriver_win32/chromedriver.exe' #chromedriver 위치
url = 'https://www.instagram.com/explore/'

ID = '' #인스타 id
PW = '' #인스타 비밀번호

In [3]:
def select_first(driver):
    first = driver.find_element_by_css_selector('._9AhH0')
    first.click()
    time.sleep(1)

In [4]:
images = []
contents = []
places = []
tags = []

data = {'image': images,
        'content': contents,
       'place': places,
       'tag': tags,
       }

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    
    ###이미지
    url = driver.current_url
    driver.execute_script("window.open();") #새 탭에서 열기
    driver.switch_to.window(driver.window_handles[1])
    driver.get(url)
    time.sleep(1)
    
    html_temp = driver.page_source
    soup_temp = BeautifulSoup(html_temp, 'lxml')

    try:
        soup_temp = soup_temp.find("div", class_='KL4Bh')
        image = soup_temp.find("img")["src"] #이미지가 두 개이상인 경우도 맨 처음 이미지만 수집됩니다.
        images.append(image) #해당 게시물이 영상이어도, 페이지 내에 뜨는 다른 게시물의 첫 번째 이미지가 수집됩니다...
            
    except:
        images.append(np.nan)
        
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    
    ###내용
    content = soup.select('div.C4VMK > span')[0].text
    if content == '':
        contents.append(np.nan)
    else:
        contents.append(content)
            
            
    ###해시태그
    tag = soup.select('a.xil3i') #댓글에 있는 해시태그도 다 수집되는 것 같아요.
    tags_temp = []
    for i in tag:
        tags_temp.append(i.get_text())
        
    if tags_temp == []:
        tags.append(np.nan)
    else:
        tags.append(tags_temp)
            
            
    ###위치
    place = soup.select('div.JF9hh')[0].text
    if place == '':
        places.append(np.nan)
    else:
        places.append(place)
            
            
    return data

In [5]:
def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
    right.click()

In [6]:
driver = webdriver.Chrome(path)
driver.get(url)
time.sleep(1)

In [7]:
#login
login_id = driver.find_element_by_name('username')
login_id.send_keys(ID)
login_pw = driver.find_element_by_name('password')
login_pw.send_keys(PW)
login_pw.send_keys(Keys.RETURN)
time.sleep(3)

In [8]:
select_first(driver)

num = 10 #크롤링할 게시물 개수

for n in range(num):
    data = get_content(driver)
    time.sleep(1)
    move_next(driver)
    time.sleep(1)

In [9]:
df = pd.DataFrame.from_dict(data)

In [10]:
df

,image,content,place,tag
0,https://scontent-ssn1-1.cdninstagram.com/v/t51...,shaman's whisperlast illustration great artist...,NaN,[#artemi_video]
1,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Perspectives showing the Ibirapuera planetariu...,Parque Ibirapuera,"[#Niemeyer, #OscarNiemeyer, #OscarNiemeyerWork..."
2,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Incorniciata ❤#palermo #sicilia,NaN,"[#palermo, #sicilia, #igerssicilia, #igerspale..."
3,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Boing. Boing. Boing. Boing. #petphotography #a...,NaN,"[#petphotography, #animalphotography, #profess..."
4,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Can't wait for covid to be over so I can go ba...,NaN,NaN
5,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Waking up above the clouds ⛅️,"Appenzell, Switzerland",NaN
6,https://scontent-ssn1-1.cdninstagram.com/v/t51...,ПУТЕШЕСТВИЕ В ШВЕЙЦАРИЮ 😍Сегодня число подписч...,Switzerland,"[#швейцария, #путешествия, #мир, #красотамира,..."
7,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Карелия😍Волшебные виды❤#карелия #мир #красота ...,NaN,"[#карелия, #мир, #красота, #природа, #лес, #пу..."
8,https://scontent-ssn1-1.cdninstagram.com/v/t51...,🎨-高雄市立美術館空間重塑-光間3F-禾磊建築｜梁豫漳-2020#Taiwan #kaohs...,高雄市立美術館,"[#Taiwan, #kaohsiung, #museum, #kaohsiungmuseu..."
9,https://scontent-ssn1-1.cdninstagram.com/v/t51...,"The Makedonium, a relict of Brutalism Architec...",NaN,"[#macedonia, #brutalism, #arcitecture, #archit..."
